In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
start = time.time()
very_start = time.time()

In [2]:
import numpy as np
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import cudf, cupy, time, rmm
cudf.__version__

'0.18.1'

In [3]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait, progress
from dask_cuda import LocalCUDACluster
import subprocess

In [4]:
from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

In [5]:
cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34727 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


In [6]:
features = ["text_ tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

In [7]:
%%time

def preprocessing(df):
    df['id']   = 1
    df['id']   = df['id'].cumsum()
    df['id'] = df['id'].astype('int32')

    df['reply_timestamp']   = df['reply_timestamp'].fillna(0)
    df['retweet_timestamp'] = df['retweet_timestamp'].fillna(0)
    df['retweet_with_comment_timestamp'] = df['retweet_with_comment_timestamp'].fillna(0)
    df['like_timestamp']    = df['like_timestamp'].fillna(0)

    df['reply_timestamp']   = df['reply_timestamp'].astype('int32')
    df['retweet_timestamp'] = df['retweet_timestamp'].astype('int32')
    df['retweet_with_comment_timestamp'] = df['retweet_with_comment_timestamp'].astype('int32')
    df['like_timestamp']    = df['like_timestamp'].astype('int32')

    df['tweet_timestamp']         = df['tweet_timestamp'].astype( np.int32 )
    df['engaged_with_user_follower_count']  = df['engaged_with_user_follower_count'].astype( np.int32 )
    df['engaged_with_user_following_count'] = df['engaged_with_user_following_count'].astype( np.int32 )
    df['engaged_with_user_account_creation']= df['engaged_with_user_account_creation'].astype( np.int32 )
    df['enaging_user_follower_count']  = df['enaging_user_follower_count'].astype( np.int32 )
    df['enaging_user_following_count'] = df['enaging_user_following_count'].astype( np.int32 )
    df['enaging_user_account_creation']= df['enaging_user_account_creation'].astype( np.int32 )

    df, = dask.persist(df)
    _ = wait(df)

    return df


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [ ]:
%%time

path = "/dataset/twitter_raw"
file_list = os.listdir(path)
file_list = sorted(file_list)
NP = 16

for file_name in tqdm(file_list):
    df = dask_cudf.read_csv(f'{path}/{file_name}', sep='\x01', header=None, names=features)
    df = df.repartition(npartitions=NP)
    df = preprocessing(df)
    df, = dask.persist(df)
    _ = wait(df)

    df.to_parquet(f'/dataset/train/{file_name}', write_index=False)


  8%|▊         | 23/291 [03:07<36:33,  8.19s/it]